# CFRM 521 - Final Report

#### By: Wooseok (Jeff), Max, Steve, Ilse, Jasmine

----

# Introduction

# Data

In [ ]:
# Load the packages (at the end we can consolidate all the packages we individually used here so that we can create a single environment.yml file for a conda environment at the end)
import numpy as np
import pandas as pd

# Read in the data (this section should be the same for everyone)


----

# Wooseok's Model

----

# Max's Model

----

# Steve's Model

----

# Ilse's Model

----

# Jasmine's Model

----

# Conclusions